In [1]:
import pandas as pd
#from fbprophet import Prophet
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.preprocessing import MinMaxScaler

In [2]:
source_df = pd.read_csv('input/raw/datafill.csv',index_col=0)
# source_df['2018/1/1 0:00':'2018/4/1 0:00']

## extract seasons and years data

In [3]:
def divide_into_seasons(df,years=[2015,2016,2017,2018],path='input/raw/seasons/'):
    for year in years:
#         print(year)
        if(year == 2018):
            df['2018/1/1 0:00':'2018/3/31 23:00'].to_csv(path+f'{year}_spring.csv')
        else:
            df[f'{year}/1/1 0:00':f'{year}/3/31 23:00'].to_csv(path+f'{year}_spring.csv')
            df[f'{year}/4/1 0:00':f'{year}/6/30 23:00'].to_csv(path+f'{year}_summer.csv')
            df[f'{year}/7/1 0:00':f'{year}/9/30 23:00'].to_csv(path+f'{year}_autumn.csv')
            df[f'{year}/10/1 0:00':f'{year}/12/31 23:00'].to_csv(path+f'{year}_winter.csv')
def divide_into_years(df,years=[2015,2016,2017],path='input/raw/years/'):
    for year in years:
        df[f'{year}/1/1 0:00':f'{year}/12/31 23:00'].to_csv(path+f'{year}_year.csv')
            
divide_into_seasons(source_df)
divide_into_years(source_df)

## generate seq2seq datasets

In [4]:
feature_map = {'ds':'TIME','y':'zhexi_in','x1':'xiaoxi_out','x2':'lengshuijiang_add','x3':'xinhua_add','x4':'zhexi_add'}

def feature_mapping(df,feature_map):
    for key,value in feature_map.items():
        df[key] = df[value]
    df['ds'] = pd.to_datetime(df['ds'])
    return df.drop(feature_map.values(),axis=1)

def make_former_Npoints_time_series(df,n):
    for i in range(1,n+1):
        df[f'current_before_{n-i+1}'] = df['y'].shift(n-i+1)
    return df.dropna()
# make_former_Npoints_time_series(ts,N)

def make_next_Mpoints_time_series(df,m):
    for i in range(1,m):
        df[f'current_next_{i}'] = df['y'].shift(-i)
    return df.dropna()
# make_next_Mpoints_time_series(ts,M)
# df = feature_mapping(source_df,feature_map)

In [5]:
N = 24
M = 6

In [8]:
df = pd.read_csv('input/raw/seasons/2015_spring.csv')
df = feature_mapping(df,feature_map)
df = pd.DataFrame(df['y'])
df = make_next_Mpoints_time_series(df,M)
df = make_former_Npoints_time_series(df,N)
df = pd.concat((df[df.columns[M:]],df[df.columns[:M]]),axis=1)
df.head(2)

,current_before_24,current_before_23,current_before_22,current_before_21,current_before_20,current_before_19,current_before_18,current_before_17,current_before_16,current_before_15,...,current_before_4,current_before_3,current_before_2,current_before_1,y,current_next_1,current_next_2,current_next_3,current_next_4,current_next_5
24,180.475,206.634,200.758,183.565,167.91,179.350,162.501,148.392,170.659,152.862,...,152.210,125.898,110.184,127.582,145.932,135.259,113.138,146.560,156.628,140.187
25,206.634,200.758,183.565,167.910,179.35,162.501,148.392,170.659,152.862,159.296,...,125.898,110.184,127.582,145.932,135.259,113.138,146.560,156.628,140.187,144.081


In [9]:
import glob
def generate_seq2seq_data(df,n,m,save_path=None):
    df = feature_mapping(df,feature_map)
    df = pd.DataFrame(df['y'])
    df = make_next_Mpoints_time_series(df,m)
    df = make_former_Npoints_time_series(df,n)
    df = pd.concat((df[df.columns[m:]],df[df.columns[:m]]),axis=1)
    if(save_path != None):
        df.to_csv(save_path)
    return df
def get_raw_csv(path):
    return glob.glob(path+'*.csv')
# get_raw_csv('input/raw/seasons/')
get_raw_csv('input/raw/years/')

['input/raw/years/2015_year.csv',
 'input/raw/years/2016_year.csv',
 'input/raw/years/2017_year.csv']

In [10]:
raw_csv_files = get_raw_csv('input/raw/years/')
for raw_csv_file in raw_csv_files:
    df = pd.read_csv(raw_csv_file)
#     print(df.columns)
    name = raw_csv_file.split('/')[-1]
#     print(name)
    df = generate_seq2seq_data(df,N,M,'input/prepared/years/'+name)
    print(name)
    print(df.shape,'\n')

2015_year.csv
(8731, 30) 

2016_year.csv
(8751, 30) 

2017_year.csv
(8732, 30) 



In [11]:
raw_csv_files = get_raw_csv('input/raw/seasons/')
for raw_csv_file in raw_csv_files:
    df = pd.read_csv(raw_csv_file)
#     print(df.columns)
    name = raw_csv_file.split('/')[-1]
#     print(name)
    df = generate_seq2seq_data(df,N,M,'input/prepared/seasons/'+name)
    print(name)
    print(df.shape,'\n')

2015_winter.csv
(2179, 30) 

2015_spring.csv
(2131, 30) 

2016_autumn.csv
(2176, 30) 

2015_summer.csv
(2155, 30) 

2017_winter.csv
(2181, 30) 

2016_spring.csv
(2155, 30) 

2016_summer.csv
(2154, 30) 

2017_summer.csv
(2154, 30) 

2017_spring.csv
(2131, 30) 

2016_winter.csv
(2179, 30) 

2018_spring.csv
(2131, 30) 

2015_autumn.csv
(2179, 30) 

2017_autumn.csv
(2179, 30) 

